# 从零实现多层感知机分类

多层感知机和多类逻辑分类很类似，就是在之间加入隐层以提升模型精度

<img src="http://zh.gluon.ai/_images/multilayer-perceptron.png" width="500">

## 导入库函数

In [1]:
import mxnet as mx
import numpy as np

from mxnet import nd
from mxnet import autograd
from mxnet import gluon
mx.random.seed(1)

## 加载数据

In [2]:
def transform(data, label):
    return data.astype(np.float32) / 255, label.astype(np.float32)
    
fashion_mnist_train = gluon.data.vision.FashionMNIST(train=True, transform=transform)
fashion_mnist_test = gluon.data.vision.FashionMNIST(train=False, transform=transform)

In [3]:
batch_size = 128
train_data = gluon.data.DataLoader(fashion_mnist_train, batch_size=batch_size, shuffle=True)
test_data = gluon.data.DataLoader(fashion_mnist_test, batch_size=batch_size, shuffle=False)

In [4]:
for (data, label) in test_data:
    print(data.shape)
    print(label.shape)
    break

(128, 28, 28, 1)
(128,)


## 参数初始化

### 小随机数初始化(来自cs231n笔记)

### <font color="red">目前最好的权重初始化方法就是Kaiming He提出的Xaiver初始化方法<br>

<font color="red">**错误**</font>：全零初始化。让我们从应该避免的错误开始。在训练完毕后，虽然不知道网络中每个权重的最终值应该是多少，但如果数据经过了恰当的归一化的话，就可以假设所有权重数值中大约一半为正数，一半为负数(**因为用零均值法中心化后数据的均值为0**)。这样，一个听起来蛮合理的想法就是把这些权重的初始值都设为0吧，因为在期望上来说0是最合理的猜测。这个做法错误的！因为如果网络中的每个神经元都计算出同样的输出，然后它们就会在反向传播中计算出同样的梯度，从而进行同样的参数更新。换句话说，如果权重被初始化为同样的值，神经元之间就失去了不对称性的源头。

<font color="red">**小随机数初始化**</font>。因此，权重初始值要非常接近0又不能等于0。解决方法就是将权重初始化为很小的数值，以此来打破对称性。其思路是：如果神经元刚开始的时候是随机且不相等的，那么它们将计算出不同的更新，并将自身变成整个网络的不同部分。小随机数权重初始化的实现方法是：$W = 0.01 * np.random.randn(D,H)$。其中randn函数是基于零均值和标准差的一个高斯分布（译者注：国内教程一般习惯称均值参数为期望$\mu$）来生成随机数的。根据这个式子，每个神经元的权重向量都被初始化为一个随机向量，而这些随机向量又服从一个多变量高斯分布，这样在输入空间中，所有的神经元的指向是随机的。也可以使用均匀分布生成的随机数，但是从实践结果来看，对于算法的结果影响极小。

<font color="red">**警告**</font>。并不是小数值一定会得到好的结果。例如，一个神经网络的层中的权重值很小，那么在反向传播的时候就会计算出非常小的梯度（因为梯度与权重值是成比例的）。这就会很大程度上减小反向传播中的“梯度信号”，在深度网络中，就会出现问题。

<font color="red">**使用1/sqrt(n)校准方差**</font>。上面做法存在一个问题，随着输入数据量的增长，随机初始化的神经元的输出数据的分布中的方差也在增大,记输出结果为$S$，那么具体推导如下：

\begin{split}
  & Var(s) = Var(\sum_i^n w_ix_i) \\ 
= & \sum_i^n Var(w_ix_i) \\ 
= & \sum_i^n[E(w_i)]^2 Var(x_i) + [E(x_i)]^2 Var(w_i) + Var(x_i)Var(w_i) \\ 
= & \sum_i^n Var(x_i)Var(w_i) \\ 
= & (nVar(w))Var(w) \end{split}

因此，如果想要$s$和$x$有一样的方差，则应该有$nVar(w) = 1$，则有$nVar(w) = nVar(aw^\prime) = n \times a^2Var(w^\prime) = 1$，则有$a = \sqrt{1/n}$，其中其中$w^\prime$为方差规范化后的参数，这便是著名的**Xaiver**初始化。

于是得出$w = np.random.randn(n) / sqrt(n)$.这样可以保持输出和输入的权重方差保持不变。

### <font color="red">改进

* Glorot等在论文[Understanding the difficulty of training deep feedforward neural networks](http://202.119.95.70/cache/4/03/proceedings.mlr.press/c896b216aca8427f10edb48249b207d1/glorot10a.pdf)中作出了类似的分析。在论文中，作者推荐初始化公式为 $\text{Var}(w) = 2/(n_{in} + n_{out}) $，其中\(n_{in}, n_{out}\)是在前一层和后一层中单元的个数。这是基于妥协和对反向传播中梯度的分析得出的结论。<br>
   
* Kaiming He等人在[Delving Deep into Rectifiers: Surpassing Human-Level Performance on ImageNet Classification](https://arxiv.org/abs/1502.01852)一文中给出了一种针对ReLU神经元的特殊初始化，并给出结论：网络中神经元的方差应该是$2.0/n$。代码为$w = np.random.randn(n) * sqrt(n/2.0)$。这个形式是神经网络算法使用ReLU神经元时的当前最佳推荐。

<font color="red">**稀疏初始化**</font>（Sparse initialization）。另一个处理非标定方差的方法是将所有权重矩阵设为0，但是为了打破对称性，每个神经元都同下一层固定数目的神经元随机连接（其权重数值由一个小的高斯分布生成）。一个比较典型的连接数目是10个。

<font color="red">**偏置（biases）的初始化**</font>。通常将偏置初始化为0，这是因为随机小数值权重矩阵已经打破了对称性。对于ReLU非线性激活函数，有研究人员喜欢使用如0.01这样的小数值常量作为所有偏置的初始值，这是因为他们认为这样做能让所有的ReLU单元一开始就激活，这样就能保存并传播一些梯度。然而，这样做是不是总是能提高算法性能并不清楚（有时候实验结果反而显示性能更差），所以通常还是使用0来初始化偏置参数。

### <font color="red">实践

当前的推荐是使用ReLU激活函数，并且使用$w = np.random.randn(n) * sqrt(n/2.0)$来进行权重初始化。

In [5]:
num_examples = 60000
num_inputs = 784
num_outputs = 10
num_hidden = 256

weight_scale = .01

# 小随机数初始化
# W1 = nd.random.normal(shape=(num_inputs, num_hidden), scale=weight_scale)
# b1 = nd.random.normal(shape=num_hidden)

# W2 = nd.random.normal(shape=(num_hidden, num_outputs), scale=weight_scale)
# b2 = nd.random.normal(shape=num_outputs)

# He Xaiver初始化
W1 = nd.random.normal(shape=(num_inputs, num_hidden)) / np.sqrt(num_inputs / 2)
b1 = nd.random.normal(shape=num_hidden)

W2 = nd.random.normal(shape=(num_hidden, num_outputs)) / np.sqrt(num_hidden / 2)
b2 = nd.random.normal(shape=num_outputs)

vs = []
params = [W1, b1, W2, b2]
for param in params:
    vs.append(param.zeros_like())
    param.attach_grad()


## 定义模型

In [6]:
def relu(ylinear):
    return nd.maximum(ylinear, 0)

In [7]:
def net(X):
    X = X.reshape((-1, 784))
    h1 = nd.dot(X, W1) + b1
    h1_activate = relu(h1)
    output = nd.dot(h1_activate, W2) + b2
    return output 

In [8]:
softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss()

In [9]:
def SGD(params, lr):
    for param in params:
        param[:] = param - lr * param.grad
        
# momentum
def momentum_sgd(params, vs, lr, mom):
    for (param, v) in zip(params, vs):
        v[:] = mom * v + lr * param.grad / batch_size
        param[:] = param -  v

## 定义评估函数

In [10]:
def evaluate_accuracy(img_iter, net):
    numerator = 0.0
    denominator = 0.0
    for i, (data, label) in enumerate(img_iter):
        output = net(data)
        prediction = nd.argmax(output, axis=1)
        numerator += nd.sum(prediction == label) 
        denominator += data.shape[0]
    return (numerator / denominator).asscalar()

In [11]:
evaluate_accuracy(test_data, net)

0.1

## 训练

In [12]:
epochs = 100
learning_rate = 0.1

for epoch in range(epochs):
    cumulative_loss = 0.0
    for i, (data, label) in enumerate(train_data):
        with autograd.record():
            output = net(data)
            loss = softmax_cross_entropy(output, label)
        loss.backward()
        SGD(params, learning_rate / batch_size)
        # momentum_sgd(params, vs, learning_rate, mom=0.9)
        cumulative_loss += nd.sum(loss).asscalar()
    
    train_acc = evaluate_accuracy(train_data, net)
    test_acc = evaluate_accuracy(test_data, net)
    print("Epoch %s, Avg train loss %s, Train acc %s, Test acc %s" 
          % (epoch, cumulative_loss / num_examples, train_acc, test_acc))

Epoch 0, Avg train loss 0.648661682765, Train acc 0.827833, Test acc 0.8307
Epoch 1, Avg train loss 0.460383709971, Train acc 0.8517, Test acc 0.8523
Epoch 2, Avg train loss 0.41622438345, Train acc 0.861067, Test acc 0.8611
Epoch 3, Avg train loss 0.392146313858, Train acc 0.871, Test acc 0.8699
Epoch 4, Avg train loss 0.370963144207, Train acc 0.874267, Test acc 0.8688
Epoch 5, Avg train loss 0.357209223207, Train acc 0.87885, Test acc 0.8724
Epoch 6, Avg train loss 0.346458313719, Train acc 0.877867, Test acc 0.8744
Epoch 7, Avg train loss 0.333545173422, Train acc 0.8841, Test acc 0.8745
Epoch 8, Avg train loss 0.325957826233, Train acc 0.890733, Test acc 0.8798
Epoch 9, Avg train loss 0.316037964598, Train acc 0.892517, Test acc 0.8798
Epoch 10, Avg train loss 0.309185288302, Train acc 0.895133, Test acc 0.8838
Epoch 11, Avg train loss 0.301318749475, Train acc 0.89905, Test acc 0.8828
Epoch 12, Avg train loss 0.295224978733, Train acc 0.8964, Test acc 0.883
Epoch 13, Avg train lo